In [1]:
import datetime
from IPython.display import JSON
import pandas as pd
import requests
import API_keys
import awswrangler as wr
import sqlalchemy

In [2]:
def get_cities_dataframe():
    cities = pd.read_csv('data/cities.csv')
    return cities[:1]

In [3]:
def get_jsons_from_AerodataboxAPI(icao):
    today = datetime.date.today()
    tomorrow = str(today + datetime.timedelta(days=1))
    
    times = [["00:00","11:59"],["12:00","23:59"]]
    jsons = []
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': API_keys.Airports_key
            }

        jsons.append(requests.request("GET", url, headers=headers, params=querystring).json())
    return jsons

In [8]:
def create_dictionary_from_json(jsons):
    dictionary = {"From_city":[],"Arrival_Local_time":[], "Arrival_UTC_time":[],"Terminal":[],"Airline":[],"Flight_number":[],"Aircraft_model":[]}
    for j in jsons:
        for flight in j['arrivals']:
            try:
                dictionary["From_city"].append(flight['departure']["airport"]['name'])
            except:
                dictionary["From_city"].append("Unknown")
            try:
                dictionary["Arrival_Local_time"].append(flight['arrival']["scheduledTimeLocal"][:-6])
            except:
                dictionary["Arrival_Local_time"].append("Unknown")
            try:
                dictionary["Arrival_UTC_time"].append(flight['arrival']["scheduledTimeUtc"][:-1])
            except:
                dictionary["Arrival_UTC_time"].append("Unknown")
            try:
                dictionary["Terminal"].append(flight['arrival']["terminal"])
            except:
                dictionary["Terminal"].append("Unknown")
            try:
                dictionary["Airline"].append(flight['airline']["name"])
            except:
                dictionary["Airline"].append("Unknown")
            try:
                dictionary["Flight_number"].append(flight['number'])
            except:
                dictionary["Flight_number"].append("Unknown")
            try:
                dictionary["Aircraft_model"].append(flight['aircraft']["model"])
            except:
                dictionary["Aircraft_model"].append("Unknown")
    return dictionary

In [9]:
def create_dataframe_from_dictionaries(dictionary):
    flights = pd.DataFrame.from_dict(dictionary)
    return flights

In [10]:
def get_all_info(cities):
    big_table = pd.DataFrame()
    for index, city in cities.iterrows():
        jsons = get_jsons_from_AerodataboxAPI(city['icao'])
        dictionary = create_dictionary_from_json(jsons)
        dataframe = create_dataframe_from_dictionaries(dictionary)
        dataframe = dataframe.assign(CityCountry = city['CityCountry'])
        dataframe = dataframe.assign(icao = city['icao'])
        dataframe["Arrival_Local_time"] = pd.to_datetime(dataframe["Arrival_Local_time"])
        dataframe["Arrival_UTC_time"] = pd.to_datetime(dataframe["Arrival_UTC_time"])
    big_table = big_table.append(dataframe,ignore_index=True)
    return big_table

In [20]:
def create_mysql_table(big_table):
    schema="gans"
    host="127.0.0.1"
    user="root"
    password=API_keys.mysqlPassword
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('flights',con=con,if_exists='append',index=False)

In [19]:
create_mysql_table(get_all_info(get_cities_dataframe()))